# Multidimensional Newton method

A simple implementation of multidimensional Newton's method and an example of its use on a nonlinear function.

In [2]:
function newtonmethod(f, Df, x₀, maxiters=20)
    # solve eqn f(x) = 0 for vector-valued x,f
    # Df is function that computes matrix of partial derivatives
    # x₀ is initial guess
    
    x = x₀
    fx = f(x)
    tol = 10*eps(float(x[1])) # crude stopping condition
    
    for n=1:maxiters
        @show x, fx         # display current x and f(x)
        
        if norm(fx) < tol   # exit if fx is small enough
            return x
        end
        
        Δx = -Df(x)\fx      # compute Newton step Δx
        
        x = x + Δx          # update x using x⁽ⁿ⁺¹⁾ =  x⁽ⁿ⁾ + Δx
        fx = f(x)           # recompute f(x) 
    end
    println("newtonmethod reached maximum iterations without converging")
    return x
end

newtonmethod (generic function with 2 methods)

## Example problem

We want to find a solution $(x,y)$ to the equations
\begin{align*}
y - x^3 &= 0 \\
x^2 + y^2 &= 1
\end{align*}

### Change of variables

To use the Newton method we need to write this in the form $f(x) = 0$ for vector $x$, $f$

\begin{align*}
 f_1(x,y) &= y - x^3 = 0 \\
 f_2(x,y) &= x^2 + y^2 - 1 = 0
\end{align*}

So we change from $(x,y)$ notation to $x = (x_1, x_2)$ and $f = (f_1, f_2)$ and define

\begin{align*}
 f(x) = \left(\begin{array}{l}
    x_2 - x_1^3 \\
    x_1^2 + x_2^2 - 1
    \end{array}\right)
\end{align*}

Given this $f(x)$, the original system of equations is now expressed as $f(x) = 0$.

### Compute matrix of partial derivatives

We need to compute $Df$, the matrix of partial derivatives, as a function of $x$,
where $\left[Df\right]_{ij} = \partial f_i / \partial x_j$. For the given $f$,

\begin{align*}
 Df(x) = 
  \left[\begin{array}{ll}
     \partial f_1 / \partial x_1 & \partial f_1 / \partial x_2\\
     \partial f_2 / \partial x_1 & \partial f_2 / \partial x_2\\
    \end{array}\right]
    = 
 \left[\begin{array}{cc}
    x_2 - 3x_1^2 & 1 \\
    2x_1 &  2 x_2
    \end{array}\right]
\end{align*} 


### Encode f(x) and Df(x) as Julia functions

In [1]:
f(x) = [x[2] - x[1]^3; 
        x[1]^2 + x[2]^2 - 1]

f (generic function with 1 method)

In [3]:
Df(x) = [-3*x[1]^2  1 ;
         2*x[1] 2*x[2]]

Df (generic function with 1 method)

In [4]:
# Test f(x) and Df(x) to be sure they work 
f([2;3])

2-element Array{Int64,1}:
 -5
 12

In [5]:
# Test f(x) to be sure it works Df([2;3])
Df([2;3])

2×2 Array{Int64,2}:
 -12  1
   4  6

### Apply Newton Method to f, Df

Use initial guess $x = (3,3)$

In [10]:
x = newtonmethod(f, Df, [0; 0])

(x, fx) = ([0, 0], [0, -1])


LoadError: [91mBase.LinAlg.LAPACKException(1)[39m

### Great, it looks like it converges. Test $f$ on the returned value of $x$

In [7]:
f(x)

2-element Array{Float64,1}:
 -1.11022e-16
  0.0        

Indeed, both components of $f(x)$ are nearly zero.